# 📊 Experiment C: Data Analysis Engine

## Objective
To ingest the JSON logs from Experiments A & B and calculate the core metrics for the paper:
1.  **Trembling Robustness ($R_{noise}$):** Derived from `intended_strategy` vs `strategy` mismatches.
2.  **Survival Rate:** How long cooperation persists.
3.  **Strategy Distribution:** Frequencies of Cooperate vs Defect.

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# Settings
pd.set_option('display.max_colwidth', None)

def load_all_results():
    files = [f for f in os.listdir('.') if f.endswith('.json') and 'benchmark' in f]
    all_data = []
    
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        for key, val in data.items():
            if 'ERROR' in key: continue
            
            # Extract metadata from key (e.g., PGG_Mistral-7B_en)
            parts = key.split('_')
            game_type = parts[0]
            model = parts[1]
            lang = parts[2]
            
            history = val['history']
            total_rounds = len(history)
            noise_count = 0
            strategies = []
            
            for r in history.values():
                for agent in r:
                    strategies.append(agent['strategy'])
                    if agent.get('is_noise'):
                        noise_count += 1
            
            all_data.append({
                "File": file,
                "Game": game_type,
                "Model": model,
                "Lang": lang,
                "Rounds": total_rounds,
                "NoiseEvents": noise_count,
                "NoiseRate": noise_count / (total_rounds * 3) if total_rounds > 0 else 0,
                "Strategies": strategies
            })
            
    return pd.DataFrame(all_data)

df = load_all_results()
if not df.empty:
    display(df)
else:
    print("No results found. Run experiments A or B first!")